In [ ]:
import moviepy.config as mconf

# Replace this path if your magick.exe is elsewhere
mconf.change_settings({
    "IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"
})


In [ ]:
import openai
from openai import OpenAI
import requests
import os
import textwrap
from moviepy.editor import *
import subprocess
import json
import glob

# API Keys
OPENAI_API_KEY = ""
# Initialize OpenAI Client
client = OpenAI(api_key=OPENAI_API_KEY)


In [4]:
def generate_script(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [ ]:
PROMPT = """You are a professional scriptwriter. Write a clear, engaging, and informative narration script for a faceless YouTube video on the topic: Artificial Intelligence.

Output only the voiceover narration in plain text. Do not include any visual scene directions, sound effects, music cues, or character actions.

Use a friendly, conversational, and informative tone.

The script should be approximately 30 seconds long.

Start with: “Hi everyone, welcome to Vivek's channel,”  
End with: “Subscribe to my channel and like my videos. Thank you.”

"""

# Generate script from LLM
script = generate_script(PROMPT)
print("📝 Generated Script:\n", script)



client = OpenAI(api_key=OPENAI_API_KEY)
response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input=script,
)

# Save the audio
with open("openai_voice.mp3", "wb") as f:
    f.write(response.content)


In [ ]:

def create_slideshow_video(images_folder, audio_path, output_path="final_video_mix11.mp4", fps=1):
    # 1. Collect images
    image_files = sorted(glob.glob(os.path.join(images_folder, "*.jpg")))
    if not image_files:
        raise FileNotFoundError("No PNG images found in folder.")

    # 2. Get audio duration
    cmd_probe = [
        "ffprobe", "-v", "error", "-select_streams", "a:0",
        "-show_entries", "stream=duration", "-of", "json", audio_path
    ]
    result = subprocess.run(cmd_probe, capture_output=True, text=True)
    audio_duration = float(json.loads(result.stdout)["streams"][0]["duration"])
    print(f"✔ Audio duration: {audio_duration:.2f}s")

    # 3. Calculate duration per image
    per_image_duration = audio_duration / len(image_files)

    # 4. Generate input.txt
    with open("input.txt", "w") as f:
        for img in image_files:
            f.write(f"file '{img}'\n")
            f.write(f"duration {per_image_duration:.2f}\n")
        f.write(f"file '{image_files[-1]}'\n")  # final image stays at end

    # 5. Run FFmpeg (ensure -vsync vfr to respect duration)
    cmd = [
        "ffmpeg", "-y",
        "-f", "concat", "-safe", "0", "-i", "input.txt",
        "-i", audio_path,
        "-vsync", "vfr",  # respect variable frame rate
        "-pix_fmt", "yuv420p",
        "-vf", "scale=iw:-2",
        "-c:v", "libx264", "-r", "24",
        "-c:a", "aac", "-shortest",
        output_path
    ]

    print("→ Running FFmpeg with variable frame rate...")
    proc = subprocess.run(cmd, capture_output=True, text=True)
    print(proc.stderr)
    proc.check_returncode()
    print(f"✅ Video created successfully: {output_path}")


In [ ]:
#31-07, 1.25
create_slideshow_video("images", "voice.mp3")


In [8]:
import requests

def download_background_music(url, filename="background_music1.mp3"):
    response = requests.get(url)
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"✔ Downloaded background music to {filename}")


In [9]:
from pydub import AudioSegment

def mix_audio(voice_path, music_path, output_path="mixed_audio1.mp3", music_volume_dB=-15):
    voice = AudioSegment.from_file(voice_path)
    music = AudioSegment.from_file(music_path).apply_gain(music_volume_dB)
    music = music[:len(voice)]  # Trim music to voice length

    mixed = voice.overlay(music)
    mixed.export(output_path, format="mp3")
    print(f"✔ Mixed audio saved to {output_path}")


In [ ]:
# Step 1: Download music
download_background_music("https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3")

# Step 2: Mix music with voice
mix_audio("openai_voice.mp3", "background_music1.mp3", output_path="mixed_audio1.mp3")

# Step 3: Create video
#create_video_with_audio_folder("images", "mixed_audio.mp3")
create_slideshow_video("images", "mixed_audio1.mp3")


In [ ]:

#Creating the subtitles for the video

def generate_and_mux_subtitles(
    video_in: str,
    audio_path: str,
    adjusted_srt: str = "adjusted_subtitles.srt",
    final_out: str = "video_with_subs.mp4"
):
    """
    1) Transcribe TTS audio → raw SRT via Whisper
    2) Auto-scale subtitle timings to match audio/video lengths
    3) Mux as soft subtitles (mov_text) into final MP4
    """
    # ——— 1) Whisper transcription → raw_srt ———
    with open("openai_voice.mp3", "rb") as audio_file:
     transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        response_format="srt"
    )

# Save the subtitle file
    raw_srt="subtitles.srt"
    with open("subtitles.srt", "w", encoding="utf-8") as f:
     f.write(transcript)
    # ——— 2) Adjust timings ———
    # get durations
    vid_info   = mediainfo(video_in)
    aud_info   = mediainfo(audio_path)
    video_dur  = float(vid_info["duration"])
    audio_dur  = float(aud_info["duration"])
    scale      = audio_dur / video_dur

    subs = pysubs2.load(raw_srt)
    for ev in subs:
        ev.start = int(ev.start * scale)
        ev.end   = int(ev.end   * scale)
    subs.save(adjusted_srt)
    print(f"✅ Adjusted subtitles saved → {adjusted_srt}")

    # ——— 3) Mux as soft subs ———
    cmd = [
        "ffmpeg", "-y",
        "-i", video_in,
        "-i", adjusted_srt,
        "-c:v", "copy",
        "-c:a", "copy",
        "-c:s", "mov_text",
        final_out
    ]
    print("→ Running:\n  " + " ".join(cmd))
    subprocess.run(cmd, check=True)
    print(f"✅ Final video with subtitles → {final_out}")


    


In [ ]:
generate_and_mux_subtitles(
        video_in="final_video_mix11.mp4",
        audio_path="openai_voice.mp3",
        adjusted_srt="adjusted_subtitles.srt",
        final_out="video_with_adjusted_subs1.mp4"
    )